# Keras에 내장된 사전 훈련 모델을 이용한 Image 분류

- ResNet50 pre-trained model 사용

- 1000 개의 class 분류 모델인 ResNet50 을 binary classifier (cat, dog) 으로 변형

- google colab 사용  

- My Drive 에 training data upload

## STEP #1: IMPORT LIBRARIES

## STEP #2: 사전 훈련된 가중치가 있는 모델 가져 오기

이 모델은 ImageNet 데이터셋에서 훈련되었습니다.

ResNet50은 컴퓨터 비전 작업에 널리 사용되는 심층 잔차 네트워크(Residual Network)의 하나로, 50개의 레이어로 구성되어 있습니다.

## STEP #3: 사전 훈련된 모델 평가

지정한 URL 에서 data 를 가져와 압축해제

**tf.keras.preprocessing.image.load_img** - image 를 PIL (Python Image Library) format 으로 load  

**tf.keras.preprocessing.image.img_to_array** - PIL image 를 numpy array 로 변경

### ResNet50 의 입력 spec 에 맞도록 Sample_Image preprocessing

tf.keras.applications.resnet50.preprocess_input() 함수는 ResNet50 모델을 사용하기 전에 이미지 데이터를 전처리합니다. 이 함수는 RGB 이미지를 BGR 형식으로 변환하고, 각 색상 채널을 중앙에 정렬하여 채널별로 zero-centering를 수행합니다. 이러한 전처리는 ResNet50 모델을 처음 훈련할 때 사용한 방식과 일치해야합니다.

### 확률 분포로 반환된 prediction 을 category name 으로 decode

### ImageDataGenerator 생성

## STEP #4: 전이학습 적용 및 model RETRAIN

tf.keras.applications.ResNet50 함수를 사용하여 include_top=False 인수를 전달함으로써, ResNet50 모델의 전결합층 (fully-connected layer, 일반적으로 모델의 마지막 부분에 위치하고 있음)을 제외한 나머지 부분을 로드할 수 있습니다. 이는 특성 추출 (feature extraction)을 위해 컨볼루션 베이스만을 사용하려는 경우에 유용하며, 이를 통해 기존 사전 학습된 모델을 새로운 문제에 맞게 조정하는 전이 학습 (transfer learning)을 수행할 수 있습니다.

여기서 weights='imagenet' 인수는 ImageNet 데이터셋에서 훈련된 가중치를 로드하라는 것을 의미합니다.

- ResNet50 의 top layer 제거 (include_top = False)  

- GlobalAaveragePooling2D + Dense() layer 5 개 추가
    - global_average_layer = keras.layers.GlobalAveragePooling2D()(base_model.output)
    - prediction_layer = keras.layers.Dense(units=2, activation='softmax')(global_average_layer)

### base model 의 top 에 5 개 layer 추가

### Fine tuning 하기 전에 새로이 add 한 Top 5 layer 들을 1 차 training 하여 초기화

### Top 50 개 layer 를 fine tuning

첫 번째 for loop는 모델의 마지막 50개 층을 제외한 모든 층의 trainable 속성을 False로 설정합니다. trainable 속성이 False로 설정된 층의 가중치는 훈련 도중 업데이트되지 않습니다. 즉, 해당 층의 가중치는 동결됩니다.

두 번째 for loop는 모델의 마지막 50개 층의 trainable 속성을 True로 설정합니다. trainable 속성이 True로 설정된 층의 가중치는 훈련 도중 업데이트됩니다. 즉, 해당 층의 가중치는 동결이 해제되고, 이 층들은 새로운 데이터에 대해 훈련됩니다.

이렇게 하면, 사전에 훈련된 ResNet50 모델의 일부 층을 동결하고 일부 층을 새로운 데이터에 대해 다시 훈련시킬 수 있습니다.

## STEP #5: MODEL 평가